In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
import warnings
import os

warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
FIGSIZE_LARGE = (16, 8)
FIGSIZE_MEDIUM = (14, 6)
FIGSIZE_SMALL = (12, 5)
DPI = 150

class ExploratoryAnalysis:
    def __init__(self, data_path: str, output_dir: str = 'Exploratory_result'):
        """Initialize EDA with dataset path and output directory."""
        self.data_path = data_path
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)
        
        # Load data
        self.df = pd.read_csv(data_path)
        self.df['date'] = pd.to_datetime(self.df['date'])
        
        if 'corrigido_temperatura' in self.df.columns:
            self.target = 'corrigido_temperatura'
        else:
            raise ValueError(
                "Target column not found!\n"
            )
        
        print(f"Dataset: {len(self.df)} rows x {len(self.df.columns)} columns")
        print(f"Period: {self.df['date'].min().date()} - {self.df['date'].max().date()}")
        print(f"Duration: {(self.df['date'].max() - self.df['date'].min()).days} days")
    
    def section_header(self, title: str):
        """Print formatted section header."""
        print(f"\n{'*' * 80}")
        print(f"  {title}")
    
    def save_plot(self, filename: str):
        """Save current plot to file."""
        filepath = os.path.join(self.output_dir, filename)
        plt.savefig(filepath, dpi=DPI, bbox_inches='tight')
        
    def basic_statistics(self):
        """Generate comprehensive basic statistics."""
        self.section_header("1. BASIC STATISTICS")
        
        # Target variable statistics
        if self.target in self.df.columns:
            print(f"TARGET VARIABLE ({self.target}):")
            print(f"  Mean:     {self.df[self.target].mean():.2f} GWh")
            print(f"  Median:   {self.df[self.target].median():.2f} GWh")
            print(f"  Std Dev:  {self.df[self.target].std():.2f} GWh")
            print(f"  Min:      {self.df[self.target].min():.2f} GWh")
            print(f"  Max:      {self.df[self.target].max():.2f} GWh")
            print(f"  Range:    {self.df[self.target].max() - self.df[self.target].min():.2f} GWh")
            print(f"  CV:       {(self.df[self.target].std() / self.df[self.target].mean()) * 100:.2f}%")
        
    
        # Missing values
        missing = self.df.isnull().sum()
        missing = missing[missing > 0].sort_values(ascending=False)
        if len(missing) > 0:
            for col, count in missing.head(20).items():
                pct = (count / len(self.df)) * 100
                print(f"  {col}: {count} ({pct:.2f}%)")
        else:
            print("No missing values found!")
        
        # Data types
        type_counts = self.df.dtypes.value_counts()
        for dtype, count in type_counts.items():
            print(f"  {dtype}: {count} columns")
        
        # Numerical columns summary
        numeric_cols = self.df.select_dtypes(include=[np.number]).columns
        print(f"\nNUMERICAL FEATURES: {len(numeric_cols)} columns")
        return self.df.describe()
    

    def temporal_analysis(self):
        """Analyze temporal patterns and trends."""
        self.section_header("3. TEMPORAL ANALYSIS")
        
        # Create temporal features
        self.df['year'] = self.df['date'].dt.year
        self.df['month'] = self.df['date'].dt.month
        self.df['day_of_week'] = self.df['date'].dt.dayofweek
        self.df['day_name'] = self.df['date'].dt.day_name()
        self.df['is_weekend'] = self.df['day_of_week'].isin([5, 6])
        
        # Time series plot
        fig, axes = plt.subplots(2, 1, figsize=FIGSIZE_LARGE, sharex=True)
        
        # Raw time series
        axes[0].plot(self.df['date'], self.df[self.target], alpha=0.7, linewidth=0.8)
        axes[0].set_title('Daily Energy Consumption Time Series', fontsize=14, fontweight='bold')
        axes[0].set_ylabel('Consumption (GWh)', fontsize=12)
        axes[0].grid(True, alpha=0.3)
        
        # Rolling means
        rolling_30 = self.df[self.target].rolling(window=30, center=True).mean()
        rolling_365 = self.df[self.target].rolling(window=365, center=True).mean()
        
        axes[1].plot(self.df['date'], rolling_30, label='30-day MA', linewidth=1.5)
        axes[1].plot(self.df['date'], rolling_365, label='365-day MA', linewidth=2)
        axes[1].set_title('Moving Averages', fontsize=14, fontweight='bold')
        axes[1].set_xlabel('Date', fontsize=12)
        axes[1].set_ylabel('Consumption (GWh)', fontsize=12)
        axes[1].legend(loc='best')
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        self.save_plot('target_timeseries.png')
        plt.close()
        
        # Yearly statistics
        print("YEARLY STATISTICS:")
        yearly_stats = self.df.groupby('year')[self.target].agg(['mean', 'std', 'min', 'max', 'count'])
        print(yearly_stats.round(2))
        
        # Monthly pattern
        print("\nMONTHLY PATTERN:")
        monthly_avg = self.df.groupby('month')[self.target].mean().sort_values(ascending=False)
        month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                       'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        for month, value in monthly_avg.items():
            print(f"  {month_names[month-1]}: {value:.2f} GWh")
        
        # Weekly pattern
        print("\nWEEKLY PATTERN:")
        weekly_avg = self.df.groupby('day_name')[self.target].mean()
        day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        for day in day_order:
            if day in weekly_avg.index:
                print(f"  {day:10s}: {weekly_avg[day]:.2f} GWh")
        
        # Weekend vs Weekday
        weekend_avg = self.df[self.df['is_weekend']][self.target].mean()
        weekday_avg = self.df[~self.df['is_weekend']][self.target].mean()
        print(f"\n  Weekday average: {weekday_avg:.2f} GWh")
        print(f"  Weekend average: {weekend_avg:.2f} GWh")
        print(f"  Weekend reduction: {((weekday_avg - weekend_avg) / weekday_avg * 100):.2f}%")
        
        # Monthly boxplot 
        fig, ax = plt.subplots(figsize=FIGSIZE_MEDIUM)
        sns.boxplot(data=self.df, x='month', y=self.target, ax=ax, palette='Set3')
        ax.set_title('Monthly Consumption Distribution', fontsize=14, fontweight='bold')
        ax.set_xlabel('Month', fontsize=12)
        ax.set_ylabel('Consumption (GWh)', fontsize=12)
        ax.set_xticklabels(month_names)
        plt.tight_layout()
        self.save_plot('monthly_consumption.png')
        plt.close()
        
        # Weekly pattern plot
        fig, ax = plt.subplots(figsize=(12, 6))
        weekly_data = self.df.groupby('day_of_week')[self.target].mean()
        ax.bar(range(7), weekly_data.values, color=['#3498db']*5 + ['#e74c3c']*2)
        ax.set_xticks(range(7))
        ax.set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
        ax.set_title('Average Consumption by Day of Week', fontsize=14, fontweight='bold')
        ax.set_ylabel('Consumption (GWh)', fontsize=12)
        ax.grid(True, alpha=0.3, axis='y')
        
        # Add values on bars
        for i, v in enumerate(weekly_data.values):
            ax.text(i, v + 1, f'{v:.1f}', ha='center', va='bottom', fontweight='bold')
        
        plt.tight_layout()
        self.save_plot('weekly_consumption.png')
        plt.close()
    
    def distribution_analysis(self):
        """Analyze distribution of target variable."""
        self.section_header("4. DISTRIBUTION ANALYSIS")
        
        # Normality test
        sample_size = min(5000, len(self.df[self.target].dropna()))
        statistic, p_value = stats.shapiro(self.df[self.target].dropna().sample(sample_size))
        print(f"NORMALITY TEST (Shapiro-Wilk):")
        print(f"  Statistic: {statistic:.4f}")
        print(f"  P-value: {p_value:.4f}")
        if p_value < 0.05:
            print(f"  Result: Data is NOT normally distributed")
        else:
            print(f"  Result: Data is normally distributed")
        
        # Skewness and Kurtosis
        skew = stats.skew(self.df[self.target].dropna())
        kurt = stats.kurtosis(self.df[self.target].dropna())
        print(f"\nSHAPE METRICS:")
        print(f"  Skewness: {skew:.4f} ", end='')
        if abs(skew) < 0.5:
            print("(Fairly symmetric)")
        elif skew > 0:
            print("(Right-skewed)")
        else:
            print("(Left-skewed)")
        
        print(f"  Kurtosis: {kurt:.4f} ", end='')
        if abs(kurt) < 0.5:
            print("(Normal tails)")
        elif kurt > 0:
            print("(Heavy tails)")
        else:
            print("(Light tails)")
        
        # Distribution plot
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        # Histogram with KDE
        axes[0, 0].hist(self.df[self.target].dropna(), bins=50, alpha=0.7, 
                       color='skyblue', edgecolor='black')
        axes[0, 0].axvline(self.df[self.target].mean(), color='red', linestyle='--', 
                          linewidth=2, label=f'Mean: {self.df[self.target].mean():.1f}')
        axes[0, 0].axvline(self.df[self.target].median(), color='green', linestyle='--', 
                          linewidth=2, label=f'Median: {self.df[self.target].median():.1f}')
        axes[0, 0].set_title('Distribution with Mean/Median', fontsize=12, fontweight='bold')
        axes[0, 0].set_xlabel('Consumption (GWh)')
        axes[0, 0].set_ylabel('Frequency')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # KDE plot
        self.df[self.target].dropna().plot(kind='kde', ax=axes[0, 1], linewidth=2, color='darkblue')
        axes[0, 1].set_title('Kernel Density Estimate', fontsize=12, fontweight='bold')
        axes[0, 1].set_xlabel('Consumption (GWh)')
        axes[0, 1].set_ylabel('Density')
        axes[0, 1].grid(True, alpha=0.3)
        
        # Q-Q plot
        stats.probplot(self.df[self.target].dropna(), dist="norm", plot=axes[1, 0])
        axes[1, 0].set_title('Q-Q Plot (Normal Distribution)', fontsize=12, fontweight='bold')
        axes[1, 0].grid(True, alpha=0.3)
        
        # Box plot
        axes[1, 1].boxplot(self.df[self.target].dropna(), vert=True, patch_artist=True,
                          boxprops=dict(facecolor='lightblue', color='navy'),
                          medianprops=dict(color='red', linewidth=2),
                          whiskerprops=dict(color='navy'),
                          capprops=dict(color='navy'))
        axes[1, 1].set_title('Box Plot', fontsize=12, fontweight='bold')
        axes[1, 1].set_ylabel('Consumption (GWh)')
        axes[1, 1].grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        self.save_plot('target_distribution.png')
        plt.close()
    
    def outlier_analysis(self):
        """Comprehensive outlier detection and analysis."""
        self.section_header("5. OUTLIER ANALYSIS")
        
        # IQR method
        Q1 = self.df[self.target].quantile(0.25)
        Q3 = self.df[self.target].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers_low = self.df[self.df[self.target] < lower_bound]
        outliers_high = self.df[self.df[self.target] > upper_bound]
        total_outliers = len(outliers_low) + len(outliers_high)
        
        print(f"IQR METHOD (1.5 × IQR):")
        print(f"  Q1 (25th percentile): {Q1:.2f} GWh")
        print(f"  Q3 (75th percentile): {Q3:.2f} GWh")
        print(f"  IQR: {IQR:.2f} GWh")
        print(f"  Lower bound: {lower_bound:.2f} GWh")
        print(f"  Upper bound: {upper_bound:.2f} GWh")
        print(f"\n  Low outliers:  {len(outliers_low):3d} ({len(outliers_low)/len(self.df)*100:.2f}%)")
        print(f"  High outliers: {len(outliers_high):3d} ({len(outliers_high)/len(self.df)*100:.2f}%)")
        print(f"  Total:         {total_outliers:3d} ({total_outliers/len(self.df)*100:.2f}%)")
        
        # Z-score method
        z_scores = np.abs(stats.zscore(self.df[self.target].dropna()))
        outliers_z = len(z_scores[z_scores > 3])
        print(f"\nZ-SCORE METHOD (|z| > 3):")
        print(f"  Outliers: {outliers_z} ({outliers_z/len(self.df)*100:.2f}%)")
        
        # Outlier visualization
        plt.figure(figsize=(14, 6))
        
        plt.plot(self.df['date'], self.df[self.target], alpha=0.6, label='Consumption')
        plt.axhline(y=lower_bound, color='red', linestyle='--', 
                   label=f'Lower bound ({lower_bound:.0f} GWh)')
        plt.axhline(y=upper_bound, color='red', linestyle='--', 
                   label=f'Upper bound ({upper_bound:.0f} GWh)')
        plt.scatter(outliers_low['date'], outliers_low[self.target], 
                   color='red', s=50, label='Low outliers', zorder=5)
        plt.scatter(outliers_high['date'], outliers_high[self.target], 
                   color='orange', s=50, label='High outliers', zorder=5)
        
        plt.title('Consumption with Outliers Highlighted', fontsize=14, fontweight='bold')
        plt.xlabel('Date', fontsize=12)
        plt.ylabel('Consumption (GWh)', fontsize=12)
        plt.legend()
        plt.grid(alpha=0.3)
        plt.tight_layout()
        
        self.save_plot('outliers_analysis.png')
        plt.close()
    
    def correlation_analysis(self):
        """Analyze correlations between features."""
        self.section_header("6. CORRELATION ANALYSIS")
        
        # Select numeric columns
        numeric_cols = self.df.select_dtypes(include=[np.number]).columns.tolist()
        
        # Remove temporal features we created
        exclude_cols = ['year', 'month', 'day_of_week', 'is_weekend']
        numeric_cols = [col for col in numeric_cols if col not in exclude_cols]
        
        if self.target in numeric_cols:
            # Calculate correlation with target
            corr_with_target = self.df[numeric_cols].corr()[self.target].abs().sort_values(ascending=False)
            
            print(f"TOP 20 FEATURES CORRELATED WITH TARGET:")
            for i, (feat, corr) in enumerate(corr_with_target.head(20).items(), 1):
                print(f"  {i:2d}. {feat:40s}: {corr:.4f}")
            
            # Select top features for heatmap
            top_features = corr_with_target.head(15).index.tolist()
            
            # Correlation matrix
            corr_matrix = self.df[top_features].corr()
            
            # Heatmap
            fig, ax = plt.subplots(figsize=(14, 12))
            mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
            sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', 
                       cmap='coolwarm', center=0, square=True, linewidths=1,
                       cbar_kws={"shrink": 0.8}, ax=ax)
            ax.set_title('Correlation Heatmap - Top 15 Features', 
                        fontsize=14, fontweight='bold', pad=20)
            plt.tight_layout()
            self.save_plot('correlation_heatmap.png')
            plt.close()
    
    def weather_impact_analysis(self):
        """Analyze weather impact on consumption."""
        self.section_header("7. WEATHER IMPACT ANALYSIS")
        
        weather_cols = [col for col in self.df.columns if any(w in col.lower() 
                       for w in ['temp', 'precipitation', 'wind', 'radiation', 'humidity', 'rain', 'solar'])]
        
        if len(weather_cols) == 0:
            print("No weather columns found in dataset")
            return
        
        print(f"Found {len(weather_cols)} weather related features")
        
        # Create subplots for top weather variables
        weather_cols_filtered = [c for c in weather_cols if c != self.target][:9]
        
        n_cols = 3
        n_rows = (len(weather_cols_filtered) + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 5*n_rows))
        axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes
        
        for idx, col in enumerate(weather_cols_filtered):
            if idx < len(axes):
                valid_data = self.df[[col, self.target]].dropna()
                if len(valid_data) > 0:
                    axes[idx].scatter(valid_data[col], valid_data[self.target], 
                                    alpha=0.3, s=10, color='steelblue')
                    
                    # Add trend line
                    z = np.polyfit(valid_data[col], valid_data[self.target], 1)
                    p = np.poly1d(z)
                    x_trend = np.linspace(valid_data[col].min(), valid_data[col].max(), 100)
                    axes[idx].plot(x_trend, p(x_trend), "r--", linewidth=2, alpha=0.8)
                    
                    # Calculate correlation
                    corr = valid_data[col].corr(valid_data[self.target])
                    
                    axes[idx].set_xlabel(col.replace('_', ' ').title(), fontsize=10)
                    axes[idx].set_ylabel('Consumption (GWh)', fontsize=10)
                    axes[idx].set_title(f'{col[:30]}\nCorr: {corr:.3f}', fontsize=10, fontweight='bold')
                    axes[idx].grid(True, alpha=0.3)
        
        # Hide unused subplots
        for idx in range(len(weather_cols_filtered), len(axes)):
            axes[idx].axis('off')
        
        plt.tight_layout()
        self.save_plot('weather_impact.png')
        plt.close()
        
        # Print weather correlations
        print("\nWEATHER CORRELATIONS WITH CONSUMPTION:")
        for col in weather_cols_filtered:
            if col in self.df.columns and self.target in self.df.columns:
                corr = self.df[col].corr(self.df[self.target])
                print(f"  {col:40s}: {corr:+.4f}")

    def run_full_analysis(self):
        """Run complete EDA pipeline."""
        try:
            self.basic_statistics()    
            self.temporal_analysis()
            self.distribution_analysis()
            self.outlier_analysis()
            self.correlation_analysis()
            self.weather_impact_analysis()
            
            print("\n" + "*" * 80)
            print("EXPLORATORY DATA ANALYSIS COMPLETED SUCCESSFULLY")
            print("*" * 80)
        except Exception as e:
            print(f"\n Error during EDA: {e}")
            import traceback
            traceback.print_exc()


def main():
    """Main execution function."""
    # Analyze original dataset (base)
    DATA_PATH = "data/processed/dataset_base.csv"  
    OUTPUT_DIR = "code/Exploratory_result"
    
    print("=" * 80)
    print("EXPLORATORY DATA ANALYSIS - Energy Consumption Forecasting for Portugal Using Ensemble Machine Learning")
    print("=" * 80)
    print(f"Dataset: {DATA_PATH}")
    print(f"Output:  {OUTPUT_DIR}")
    print()
    
    try:
        eda = ExploratoryAnalysis(DATA_PATH, OUTPUT_DIR)
        eda.run_full_analysis()
    except FileNotFoundError:
        print(f"\nERROR: Dataset not found at {DATA_PATH}")
        print("\n Make sure the dataset exists:")
    except Exception as e:
        print(f"\n ERROR: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()

EXPLORATORY DATA ANALYSIS - Energy Consumption Forecasting for Portugal Using Ensemble Machine Learning
Dataset: data/processed/dataset_base.csv
Output:  code/Exploratory_result

Dataset: 5849 rows x 20 columns
Period: 2010-01-01 - 2026-01-06
Duration: 5849 days

********************************************************************************
  1. BASIC STATISTICS
TARGET VARIABLE (corrigido_temperatura):
  Mean:     137.60 GWh
  Median:   138.00 GWh
  Std Dev:  15.21 GWh
  Min:      72.00 GWh
  Max:      185.00 GWh
  Range:    113.00 GWh
  CV:       11.05%
No missing values found!
  float64: 18 columns
  datetime64[ns]: 1 columns
  int64: 1 columns

NUMERICAL FEATURES: 19 columns

********************************************************************************
  3. TEMPORAL ANALYSIS


YEARLY STATISTICS:
        mean    std    min    max  count
year                                    
2010  142.09  16.08  108.0  178.0    364
2011  138.49  15.55  101.0  180.0    365
2012  133.43  13.85   99.0  162.0    366
2013  133.85  13.17  106.0  166.0    365
2014  134.16  13.45   99.0  166.0    365
2015  134.50  13.16  100.0  166.0    365
2016  134.81  13.47  101.0  166.0    366
2017  136.49  13.49   99.0  164.0    365
2018  138.62  14.04  105.0  168.0    365
2019  138.48  14.20  104.0  169.0    365
2020  134.20  17.26   98.0  175.0    366
2021  136.16  14.49  101.0  172.0    365
2022  138.73  14.48  104.0  168.0    365
2023  139.20  15.99  100.0  178.0    365
2024  142.35  16.10  108.0  183.0    366
2025  145.73  17.26   72.0  182.0    365
2026  158.83  18.14  137.0  185.0      6

MONTHLY PATTERN:
  Jan: 156.75 GWh
  Feb: 150.45 GWh
  Dec: 147.00 GWh
  Nov: 141.20 GWh
  Mar: 139.56 GWh
  Jul: 136.58 GWh
  Sep: 133.68 GWh
  Oct: 132.10 GWh
  Apr: 129.51 GWh
  Jun: 129.27 GWh
  Au


********************************************************************************
  4. DISTRIBUTION ANALYSIS
NORMALITY TEST (Shapiro-Wilk):
  Statistic: 0.9908
  P-value: 0.0000
  Result: Data is NOT normally distributed

SHAPE METRICS:
  Skewness: 0.1453 (Fairly symmetric)
  Kurtosis: -0.1329 (Normal tails)



********************************************************************************
  5. OUTLIER ANALYSIS
IQR METHOD (1.5 × IQR):
  Q1 (25th percentile): 128.00 GWh
  Q3 (75th percentile): 146.00 GWh
  IQR: 18.00 GWh
  Lower bound: 101.00 GWh
  Upper bound: 173.00 GWh

  Low outliers:   12 (0.21%)
  High outliers:  71 (1.21%)
  Total:          83 (1.42%)

Z-SCORE METHOD (|z| > 3):
  Outliers: 2 (0.03%)

********************************************************************************
  6. CORRELATION ANALYSIS


TOP 20 FEATURES CORRELATED WITH TARGET:
   1. corrigido_temperatura                   : 1.0000
   2. shortwave_radiation_sum                 : 0.4369
   3. hidrica                                 : 0.4089
   4. temperature_2m_mean                     : 0.3905
   5. temperature_2m_max                      : 0.3897
   6. temperature_2m_min                      : 0.3617
   7. relative_humidity_2m_mean               : 0.3089
   8. exportacao                              : 0.2746
   9. gas_natural                             : 0.2396
  10. eolica                                  : 0.2392
  11. outra_termica                           : 0.1725
  12. producao_bombagem                       : 0.1537
  13. precipitation_sum                       : 0.1482
  14. importacao                              : 0.1215
  15. consumo_bombagem                        : 0.0925
  16. consumo_armazenamento                   : 0.0925
  17. wind_speed_10m_mean                     : 0.0830
  18. biomassa           


********************************************************************************
  7. WEATHER IMPACT ANALYSIS
Found 9 weather related features



WEATHER CORRELATIONS WITH CONSUMPTION:
  solar                                   : -0.0226
  temperature_2m_mean                     : -0.3905
  temperature_2m_max                      : -0.3897
  temperature_2m_min                      : -0.3617
  relative_humidity_2m_mean               : +0.3089
  precipitation_sum                       : +0.1482
  wind_speed_10m_mean                     : +0.0830
  shortwave_radiation_sum                 : -0.4369

********************************************************************************
EXPLORATORY DATA ANALYSIS COMPLETED SUCCESSFULLY
********************************************************************************
